In [69]:
import json
from pprint import pprint
import os
import glob
import re
import imp
import sys
import inspect

In [99]:
proj_dir = os.path.expanduser('/home/dimert/repos/vacancy_analysis/')

def do_skip_dir(dirname):
    exclude_dirs = ['ropeproject', 'tests']
    skipdir = False
    for excl_dir in exclude_dirs:
        if re.search(excl_dir, dirname):
            skipdir = True
            break
    return skipdir

def python_files(proj_dir):
    for dirpath, _, files in os.walk(proj_dir):
        if do_skip_dir(dirpath):
            continue
        for fname in files:
            if re.search(r'\.py$', fname) and not re.search(r'setup\.py', fname):
                yield os.path.abspath(os.path.join(dirpath, fname))
                
def get_methods(cls):   
  return [i for i in cls.__dict__.keys() if i[:2] != '__' and inspect.isfunction(cls.__dict__[i])]

def get_attrs(cls):   
  return [i for i in cls.__dict__.keys() if i[:2] != '__' and not inspect.isfunction(cls.__dict__[i])]

def get_classes_from_file(fname):
    classes = {}
    try:
        module = imp.load_source('', fname)
    except:
        return classes
    for clsname, cls in inspect.getmembers(module, inspect.isclass):
        classes[clsname] = {'attrs': get_attrs(cls), 'methods': get_methods(cls)}
    return classes

def get_classes(proj_dir):
    sys.path.append(proj_dir)
    all_classes = {}
    for fname in python_files(proj_dir):
        #print(pfile)
        classes = get_classes_from_file(fname)
        all_classes.update(classes)
    from pprint import pprint
    pprint(all_classes)
    
get_classes(proj_dir)


{'Base': {'attrs': ['metadata', '_decl_class_registry'], 'methods': []},
 'Column': {'attrs': [],
            'methods': ['append_foreign_key',
                        'references',
                        '_set_parent',
                        '_make_proxy',
                        '_on_table_attach',
                        'get_children',
                        '_setup_on_memoized_fks',
                        'copy',
                        '_compiler_dispatch']},
 'Counter': {'attrs': ['fromkeys'],
             'methods': ['most_common',
                         '_keep_positive',
                         'subtract',
                         'elements',
                         'update',
                         'copy']},
 'DBEngine': {'attrs': [],
              'methods': ['dispose',
                          'drop_database',
                          'get_url',
                          'create_db',
                          'get_session']},
 'DBManager': {'attrs': [], 'methods'

In [90]:
from copy import deepcopy


class ClassFactory():
    def __init__(self):
        self.cls_counter = 0
        with open('example_data/2classes.json') as doc:
            self.reference_json = json.load(doc)
        self.class_template = self.reference_json['stage']['objects'][0]
        self.generated_classes = []

    def add_class(self, name, attrs, methods):
        cls = deepcopy(self.class_template)
        current_id = self.cls_counter * 7
        # class self id 3 = 0
        cls['id'] = current_id
        #### Class
        child = cls['children'][0]
        # id = 1
        child['id'] = current_id + 1
        #class_id 1 = 2
        child['children'][0]['id'] = current_id + 2
        # class id 2 = 2 
        child['constraints']['constraints'][0]['HeightConstraint']['heightInfo'][0]['id'] = current_id + 2

        #### Attr
        child = cls['children'][1]
        # id = 3
        child['id'] = current_id + 3
        #class_id 1 = 4
        child['children'][0]['id'] = current_id + 4
        # class id 2 = 4
        child['constraints']['constraints'][1]['HeightConstraint']['heightInfo'][0]['id'] = current_id + 4

        child['constraints']['constraints'][0]['PositionConstraint']['nodeId'] = current_id + 1

        #### Methods
        child = cls['children'][2]
        # id = 5
        child['id'] = current_id + 5


        #class_id 1 = 6
        child['children'][0]['id'] = current_id + 6
        # class id 2 = 0
        child['constraints']['constraints'][1]['PositionConstraint']['nodeId'] = current_id + 3


        #all ids
        cls['constraints']['constraints'][0]['HeightConstraint']['heightInfo'][0]['id'] = current_id + 1
        cls['constraints']['constraints'][0]['HeightConstraint']['heightInfo'][0]['id'] = current_id + 3
        cls['constraints']['constraints'][0]['HeightConstraint']['heightInfo'][0]['id'] = current_id + 6

        class_name_html = ('<p style="text-align: center;"><span class="gliffy-placeholder-text" '
    'style="font-family: Arial; font-size: 12px; font-weight: bold; '
    'text-decoration: none; line-height: 14px; color: rgb(0, 0, '
    '0);">{}</span></p>').format(name)
        cls['children'][0]['children'][0]['graphic']['Text']['html'] = class_name_html
    
        attr_html = ('<p style="text-align: left;"><span class="gliffy-placeholder-text" style="font-family: Arial; '
                    'font-size: 12px; font-weight: normal; text-decoration: none; line-height: 14px; color: '
                     'rgb(0, 0, 0);">\n</span></p>')
        for attr in attrs:
            attr_html += ('<p style="text-align: left;"><span class="gliffy-placeholder-text" style="font-family: Arial; '
                    'font-size: 12px; font-weight: normal; text-decoration: none; line-height: 14px; color: '
                     'rgb(0, 0, 0);">{}\n</span></p>').format(attr)
        cls['children'][1]['children'][0]['graphic']['Text']['html'] = attr_html

        methods_html = ('<p style="text-align: left;"><span class="gliffy-placeholder-text" style="font-family: Arial; '
                    'font-size: 12px; font-weight: normal; text-decoration: none; line-height: 14px; color: '
                     'rgb(0, 0, 0);">\n</span></p>')
        for method in methods:
            methods_html += ('<p style="text-align: left;"><span class="gliffy-placeholder-text" style="font-family: Arial; '
                    'font-size: 12px; font-weight: normal; text-decoration: none; line-height: 14px; color: '
                     'rgb(0, 0, 0);">{}\n</span></p>').format(method)
        cls['children'][2]['children'][0]['graphic']['Text']['html'] = methods_html


        cls['x'] = 100 + self.cls_counter * 200
        self.cls_counter += 1
        self.generated_classes.append(cls)
    
    def write(self, filename):
        x['stage']['objects'] = [new_cls1, new_cls2]
        output = deepcopy(self.reference_json)
        output['stage']['objects'] = self.generated_classes
        with open(filename, 'w') as outfile:
            json.dump(output, outfile)

    
factory = ClassFactory()
factory.add_class('class1', ['attr12', 'attr11'], ['method12', 'method11'])
factory.add_class('class2', ['attr22', 'attr21'], ['method22'])
factory.add_class('class3', [], ['method22'])
factory.write('example_data/output.json')
